In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, brown
from nltk.stem import PorterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np

import gzip
import json
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Fit and transform the training data to a document-term matrix using TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error

import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

# Data Preparation

data = pd.read_json('AMAZON_FASHION.json', lines=True)
data

In [4]:
data = pd.read_json('CDs_and_Vinyl.json', lines = True)
data            

,reviewerID,asin,reviewerName,verified,reviewText,overall,reviewTime,summary,unixReviewTime,vote,style,image
0,A171I27YBM4FL6,0001393774,Eline Mann,True,I love this CD. So inspiring!,5,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
1,A1H1DL4K669VQ9,0001393774,Judith Paladino,True,Love it!! Great seller!,5,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
2,A23WIHT5886G36,0001393774,Mark Leszczynski,True,I bought this on cassette tape in the 80's. So...,5,"04 19, 2016",I bought this on cassette tape in the 80's. ...,1461024000,NaN,NaN,NaN
3,A3SZNOJP8OL26X,0001393774,mimi,True,as good as i remember back when i bought the o...,5,"03 30, 2016",Five Stars,1459296000,NaN,NaN,NaN
4,A3V5XBBT7OZG5G,0001393774,gflady,True,One of my very favourite albums from one of my...,5,"02 23, 2016",One of my very favourite albums from one of my...,1456185600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4543364,A1KCWOF28NPM3M,B01HJG3VZI,Peter G.,True,"Great 5 LP by The Guess Who, love all these al...",5,"11 4, 2017",Guess Who 5 Pack!,1509753600,NaN,{'Format:': ' Audio CD'},NaN
4543365,ASOJC2B605GDG,B01HJG3UQI,tony g.,True,Brings back many MEMORIES of a time gone bye.,5,"04 24, 2017",Five Stars,1492992000,2,{'Format:': ' Audio CD'},NaN
4543366,A13EWHP1W5X77J,B01HJG3UQI,Anthony Jones,True,Her string of albums were really enjoyable to ...,4,"12 6, 2016",Enjoyable collection of albums,1480982400,NaN,{'Format:': ' Audio CD'},NaN
4543367,A1QN6Z2E6M0S2D,B01HJG3UQI,StarTrek1701,True,"If you decide to buy this, be sure to hold on ...",5,"10 13, 2016",A great way to add to your collection,1476316800,12,{'Format:': ' Audio CD'},NaN


In [5]:
data

,reviewerID,asin,reviewerName,verified,reviewText,overall,reviewTime,summary,unixReviewTime,vote,style,image
0,A171I27YBM4FL6,0001393774,Eline Mann,True,I love this CD. So inspiring!,5,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
1,A1H1DL4K669VQ9,0001393774,Judith Paladino,True,Love it!! Great seller!,5,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
2,A23WIHT5886G36,0001393774,Mark Leszczynski,True,I bought this on cassette tape in the 80's. So...,5,"04 19, 2016",I bought this on cassette tape in the 80's. ...,1461024000,NaN,NaN,NaN
3,A3SZNOJP8OL26X,0001393774,mimi,True,as good as i remember back when i bought the o...,5,"03 30, 2016",Five Stars,1459296000,NaN,NaN,NaN
4,A3V5XBBT7OZG5G,0001393774,gflady,True,One of my very favourite albums from one of my...,5,"02 23, 2016",One of my very favourite albums from one of my...,1456185600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4543364,A1KCWOF28NPM3M,B01HJG3VZI,Peter G.,True,"Great 5 LP by The Guess Who, love all these al...",5,"11 4, 2017",Guess Who 5 Pack!,1509753600,NaN,{'Format:': ' Audio CD'},NaN
4543365,ASOJC2B605GDG,B01HJG3UQI,tony g.,True,Brings back many MEMORIES of a time gone bye.,5,"04 24, 2017",Five Stars,1492992000,2,{'Format:': ' Audio CD'},NaN
4543366,A13EWHP1W5X77J,B01HJG3UQI,Anthony Jones,True,Her string of albums were really enjoyable to ...,4,"12 6, 2016",Enjoyable collection of albums,1480982400,NaN,{'Format:': ' Audio CD'},NaN
4543367,A1QN6Z2E6M0S2D,B01HJG3UQI,StarTrek1701,True,"If you decide to buy this, be sure to hold on ...",5,"10 13, 2016",A great way to add to your collection,1476316800,12,{'Format:': ' Audio CD'},NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4543369 entries, 0 to 4543368
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   reviewerID      object
 1   asin            object
 2   reviewerName    object
 3   verified        bool  
 4   reviewText      object
 5   overall         int64 
 6   reviewTime      object
 7   summary         object
 8   unixReviewTime  int64 
 9   vote            object
 10  style           object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 385.6+ MB


In [7]:
data.shape

(4543369, 12)

In [8]:
min_amount_product_mentions = 20
min_amount_user_mentions = 20
filtered_df = data[['asin', 'reviewerID', 'overall', 'reviewText']]
    
filtered_df = filtered_df.drop_duplicates()    
filtered_df = filtered_df[filtered_df['asin'].map(filtered_df['asin'].value_counts()) >= min_amount_product_mentions]
filtered_df = filtered_df[filtered_df['reviewerID'].map(filtered_df['reviewerID'].value_counts()) >= min_amount_user_mentions]

In [9]:
df = filtered_df[~filtered_df.reviewText.isna()]
df.rename(columns={'reviewText':'sentence'}, inplace=True)
print(len(df))

453301


C:\Users\D070651\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\D070651\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\D070651\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\D070651\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [19]:
STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in nltk.word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    


In [20]:
df = clean_sentences(df)

Cleaning sentences...


C:\Users\D070651\AppData\Local\Temp/ipykernel_22804/845098326.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_sentence'] = df['sentence'].apply(clean_text)
C:\Users\D070651\AppData\Local\Temp/ipykernel_22804/845098326.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tok_lem_sentence'] = df['clean_sentence'].apply(


In [21]:
print(len(df))
df[['sentence', 'clean_sentence', 'tok_lem_sentence']]

453301


,sentence,clean_sentence,tok_lem_sentence
6,I recall loving his other albums and maybe thi...,i recall loving his other albums and maybe thi...,"[i, recall, loving, his, other, album, and, ma..."
32,Keith Green / Songs for the Shepherd: His pre...,keith green songs for the shepherd his pre...,"[keith, green, song, for, the, shepherd, his, ..."
37,Keith Green had a passionate love for Jesus. ...,keith green had a passionate love for jesus ...,"[keith, green, had, a, passionate, love, for, ..."
117,Bought to replace original I purchased many ye...,bought to replace original i purchased many ye...,"[bought, to, replace, original, i, purchased, ..."
150,I love this CD! It is always part of my Christ...,i love this cd it is always part of my christ...,"[i, love, this, cd, it, is, always, part, of, ..."
...,...,...,...
4543140,Good job,good job,"[good, job]"
4543226,Luv my Heart,luv my heart,"[luv, my, heart]"
4543228,Ann's still got it ... and I doubt very seriou...,ann still got it and i doubt very serious...,"[ann, still, got, it, and, i, doubt, very, ser..."
4543263,I first heard of these guys was when I picked ...,i first heard of these guys was when i picked ...,"[i, first, heard, of, these, guy, wa, when, i,..."


In [22]:
X = df['sentence']
y = df['overall']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# SVM

In [ ]:
word_count = CountVectorizer(stop_words = 'english')
tfidf = TfidfVectorizer(stop_words = 'english')

In [23]:
def review_classification(X, y, model,clf_model):
    
    X_c = model.fit_transform(X)
    #print(X_c)
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test =  train_test_split(X_c, y, test_size=0.3, stratify=y, random_state=42)
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    
    
    
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print("Mean Squared Error",mean_squared_error(y_test, y_pred))
    print("RMSE:", metrics.mean_squared_error(y_test, y_pred, squared=False))
    print ('Model Accuracy: {}'.format(acc))

In [ ]:
#
review_classification(X, y, word_count, SVC())

# features: 272743
# train records: 317310
# test records: 135991


In [ ]:
review_classification(X, y, tfidf, SVC())

In [ ]:
review_classification(X, y, count, MultinomialNB())

In [ ]:
review_classification(X, y, count, KNeighborsClassifier())

In [ ]:
review_classification(X, y, count, LogisticRegression())
